In [27]:
!pip install pysimplegui



     |████████████████████████████████| 509 kB 1.1 MB/s eta 0:00:01


In [50]:
import sqlite3
con=sqlite3.connect('CS281_Stage3.db')
cur=con.cursor()

for row in cur.execute('SELECT FirstName FROM User'):
    print(row)

con.close()

('Ece',)
('Naz',)
('Ahmet',)
('Arda',)
('Derin',)
('Mahmut',)
('Soda',)
('Kedi',)
('Yakamoz',)
('Ruzgar',)
('Ayse',)
('Zeynep',)
('Nazim',)
('Omer',)
('Tuna',)


In [12]:
import random
import sqlite3
import PySimpleGUI as sg
from datetime import datetime

# connect to the DB
con = sqlite3.connect('CS281_Stage3.db')
cur = con.cursor()

# global variables
login_user_id = -1
login_user_name = -1
login_user_surname= -1
auction_id = -1

# window functions

def Main():
    choices = ('Seller', 'Buyer', 'Admin')
    layout1 = [[sg.Text('Welcome to the Online Auction System! Please enter your role.')],
              [sg.Combo(choices, size=(15, len(choices)), key='chosen_role')],
              [sg.Button('OK')]]
    
    return sg.Window('Main Window', layout1)


def window_login():
    
    layout2 = [[sg.Text('Please enter your information.')],
              [sg.Text('SSN:',size =(10,1)), sg.Input(size=(10,1), key='ssn')],
              [sg.Text('Password:', size =(10,1)), sg.Input(size=(10,1), key='password')],
              [sg.Button('Login'),sg.Button('Return to Main Window')]]

    return sg.Window('Login Window', layout2)

def window_seller():
    
    layout = [[sg.Text('Welcome ' + login_user_name + login_user_surname)],
              [sg.Button('My Auctions')],
              [sg.Button('My Bills')],
              [sg.Button('Update My Profile')],
              [sg.Button('Logout')]]

    return sg.Window('Seller Window', layout)

def window_myAuctions():
    
    auctions = []
    for row in cur.execute('''SELECT AuctionID, Description, BINPrice, CurrentPrice, Status, Title FROM Auction'''):
        auctions.append(row)
        
    layout = [[sg.Text('AuctionID, Status, Title, Description')],
              [sg.Listbox(auctions, size=(100,10), key='auctionlist')],
              [sg.Button('Create Auction')],
              [sg.Button('Update Auction')],
              [sg.Button('Delete Auction')],
              [sg.Button('Show the Bids')],
              [sg.Button('Return to Seller Window')]]
    return sg.Window('My Auctions Window', layout)
    

def window_createAuction():
    
    choices = []
    for row in cur.execute('SELECT DISTINCT Cname FROM Category'):
             choices.append(row[0])
                            
    layout = [[sg.Text('Choose a category')],
               [sg.Combo(choices, size=(15, len(choices)), key='category')],
               [sg.Text('Description',size =(25,1)), sg.Input(size=(25,1), key='description')],
               [sg.Text('Buy It Now Price',size =(25,1)), sg.Input(size=(25,1), key='binprice')],
               [sg.Text('Current Price:',size =(25,1)), sg.Input(size=(25,1), key='cprice')],
               [sg.Text('Title:',size =(25,1)), sg.Input(size=(25,1), key='title')],
               [sg.Button('Add Auction')],
               [sg.Button('Return to Seller Window')]]
              
    return sg.Window('Create Auction Window', layout)


def window_billinfo(aid):
              
    cur.execute('SELECT FirstName, LastName, Bill.TransNo,NetAmount, SSSN, WSSN, Payment FROM Bill, isAbout, User WHERE isabout.AuctionID = ? AND Bill.WSSN = User.SSN AND isabout.TransNo = Bill.TransNo', (aid,))        
    row = cur.fetchone()       
    FirstName = row[0]
    LastName = row[1]
    TransNo = row[2]
    NetAmount = row[3]
    SSSN = row[4]
    WSSN = row[5]
    Payment = row[6]

    layout = [[sg.Text('Auction Bill')],
               [sg.Text('Winner Name:' + row[0]+ ' ' + row[1])],
               [sg.Text('TransNo, NetAmount, SSSN, WSSN, Payment:')],
               [sg.Text(str(row[2]) + ', ' + str(row[3]) + ', ' + str(row[4]) + ', ' + str(row[5]) + ', ' + str(row[6]))],
               [sg.Button('Return to Seller Window')]]
 
    return sg.Window('Bill Information Window', layout)              
            

def window_myBills():
     
    bills = []
    for row in cur.execute("SELECT DISTINCT AuctionID, Title, Description,Status FROM Auction WHERE Status = 'Finished' "):
        bills.append(row)
        
    layout = [[sg.Text("Select an auction to display its bill:")],
              [sg.Text("AuctionID, Title, Description, Status")],
              [sg.Listbox(bills, size=(100,10), key='billslist')],
              [sg.Button('Go to Bill')],
              [sg.Button('Return to Seller Window')]]
              
    return sg.Window('My Bills Window', layout)
 
    
def window_bids(a_id):
    
    choices = []      
    for row in cur.execute('SELECT BiddingPrice, FirstName, LastName FROM Bid, bids, have, User WHERE have.BidID = Bid.BidID AND have.AuctionID = ? AND Bid.BidID = bids.BidID AND bids.BSSN = User.SSN ORDER BY BiddingPrice DESC , FirstName, LastName', (a_id,)):
        choices.append(row)
    layout = [[sg.Text('Bids in descending order:')],
              [sg.Text('Bidding Price, Name of the Bidder:')],
              [sg.Listbox(choices, size=(100,10), key='bidslist')],
              [sg.Button('Return to Seller Window')]]
    
    return sg.Window('Auction Bids Window', layout)

def window_profile():
    
    layout = [[sg.Text('Update My Payment Information')],
              [sg.Text('Please enter your updated IBAN',size =(25,1)), sg.Input(size=(25,1), key='iban')],
              [sg.Button('Enter')],
              [sg.Button('Return to Seller Window')]]
    
    return sg.Window('Update Payment Information Window', layout)
                             
def window_updateAuction(a_id):
    
    layout = [[sg.Text('Update Auction Information')],
              [sg.Text('Please enter new title:',size =(25,1)), sg.Input(size=(25,1), key='newtitle')],
              [sg.Button('Update Title')],
              [sg.Button('Return to Seller Window')]]
    
    return sg.Window('Update Auction Information Window', layout)

                
     
def button_profile(values):
    global window
    new_iban = values['iban']  
    if not new_iban.isnumeric():
        sg.popup('IBAN should be numeric!')
    else:
        iban= new_iban[0][0]
        cur.execute('UPDATE Seller SET IBAN = ? WHERE SSSN = ?', (iban,login_user_id))
        sg.popup('Your IBAN is successfully updated. Your new iban is: ' + new_iban)           
        window.Element('iban').Update(value='')
    
        window.close()
        window = window_profile()
        
def button_update(values):

        
    global window
    selected_auction = values['auctionlist']
    if len(selected_auction) == 0:
        sg.popup('Please select an auction!')
    else:
        auction_id = selected_auction[0][0]
        new_title = values['newtitle']  
        cur.execute('UPDATE Auction SET Title = ? WHERE AuctionID = ?', (title,auction_id))
        sg.popup('Title of the auction is successfully updated. New title is: ' + new_title)           
        window.Element('title').Update(value='')
                       
        
def button_bidsInfo(values):
    
    global window
    selected_auction = values['auctionlist']
    if len(selected_auction) == 0:
        sg.popup('Please select an auction!')
    else:
        auction_id = selected_auction[0][0]
        window.close()
        window= window_bids(auction_id)
        

def button_login(values):
    
    global login_user_id 
    global login_user_name
    global login_user_surname
    global window
    
    ssn = values['ssn']
    passw = values['password']
    if ssn == '':
        sg.popup('SSN cannot be empty')
    elif passw == '':
        sg.popup('Password cannot be empty')
    else:
        cur.execute('''SELECT SSN, FirstName, 
                    LastName 
                    FROM User 
                    WHERE SSN=? 
                    AND Password = ?'''
                    ,(ssn,passw))
        row = cur.fetchone()
        
        if row is None:
            sg.popup('SSN or password is wrong!')
        else:
            login_user_id = ssn
            login_user_name = row[1]
            login_user_surname = row[2]
            
            sg.popup('Welcome, ' + str(login_user_name) + ' ' + str(login_user_surname) )
            window.close()
            window = window_seller()
            
def bill_button_function(values):
    
    global window
    selected_auction = values['billslist']
    if len(selected_auction) == 0:
        sg.popup('Please select an auction!')
    else:
        auction_id = selected_auction[0][0]
        window.close()
        window= window_billinfo(auction_id)
        
def button_createAuction(values):
    
    category = values['category']
    cat=category[0]
    description = values['description']
    binprice = values['binprice']
    curprice = values['cprice']
    title = values['title']
        
    if category == '':
        sg.popup('You should select a category!')
                  
    cur.execute('SELECT MAX(AuctionID) FROM Auction')
    row = cur.fetchone()
    new_id = row[0] + 1
    default = 0
    
    cur.execute('INSERT INTO Auction VALUES(?,?,null,null,null,?,?,null,?,?,null,?,?,null,null)',(new_id,str(description), binprice, curprice, str(title), default, cat, login_user_id)) 

    sg.popup('Auction is successfully inserted ' + str(title) + 'with id' + ' '  + str(new_id))
            
    window.Element('category').Update(value='')
    window.Element('description').Update(value='')
    window.Element('binprice').Update(value='')
    window.Element('cprice').Update(value='')
    window.Element('title').Update(value='')
                       
              
def button_role(values):
    global window
    if not values['chosen_role']:
        sg.popup('Please choose a role.')
    if values['chosen_role']=='Seller':
        window.close()
        window = window_login()
    if values['chosen_role']=='Admin' or values['chosen_role']=='Buyer':
        window.close()
        sg.popup('Close') 
        
def deleteAuction(values):  
    
    selected_auction= values['auctionlist']   
    if len(selected_auction) == 0:
        sg.popup('Please select an auction to delete')
    else:
        x = sg.popup_yes_no('Are you sure?')
        if x == "Yes":
            selected_auctionid = selected_auction[0][0]
            cur.execute("DELETE FROM Auction WHERE AuctionID = ? ", (selected_auctionid,))
            sg.popup('The auction is successfully deleted.')
            auctions = []
            for row in cur.execute('''SELECT AuctionID, Description, BINPrice, CurrentPrice, Status, Title FROM Auction'''):
                auctions.append(row)
            window.Element('auctionlist').Update(values = auctions)        
        
window = Main()              
while True:
    event, values = window.read()
    if event == 'OK':
        button_role(values)
    elif event == 'Login':
        button_login(values)
    elif event == 'Return to Main Window':
        window.close()
        window = Main()
    elif event == 'Return to Seller Window':
        window.close()
        window = window_seller()
    elif event == 'My Auctions':
        window.close()
        window = window_myAuctions()
    elif event == 'My Bills':
        window.close()
        window = window_myBills()    
    elif event == 'Create Auction':
        window.close()
        window = window_createAuction()
    elif event == 'Add Auction':
        button_createAuction(values)
        
    elif event== 'Update My Profile':
        window.close()
        window = window_profile()
        
    elif event== 'Enter':
        button_profile(values)
    
    elif event == 'Update Title':
        button_update(values)
        
    elif event == 'Update Auction':   
        window.close()
        window = window_updateAuction(values)
    
    elif event == 'Delete Auction':
        deleteAuction(values)       
    elif event == 'Show the Bids':
        button_bidsInfo(values)   
    elif event == 'Go to Bill':
        bill_button_function(values)    
    elif event == 'Add Auction':
        window.close()
        window = window_createAuction()
    elif event == 'Delete':
        window = window_deleteAuction()  
    elif event == 'Logout':
        login_user_id = -1
        login_user_name = -1
        login_user_surname= -1
        auction_id = -1
        window.close()
        window = window_login()
    elif event == sg.WIN_CLOSED:
        break
              
window.close()

con.commit()
con.close()          